In [7]:
import pandas as pd
import random

In [8]:
bros = ["Jungstein", "Max", "Walker", "Knopf", "Joe", "Thomas", "Lacy", "Jon", "Conway", "Nick"]
print(len(bros))
bros

10


['Jungstein',
 'Max',
 'Walker',
 'Knopf',
 'Joe',
 'Thomas',
 'Lacy',
 'Jon',
 'Conway',
 'Nick']

## Create matrixes for teams and people you have played against

In [47]:
def zero_out(df):
    processed_bros = []
    for _cnt, bro in enumerate(bros):
        for other_bro in bros:
            if bro == other_bro:# and other_bro not in processed_bros:
                pass # null out or leave as null?
                #df.loc[bro, other_bro] = "#
            else:
                df.loc[bro, other_bro] = 0
        processed_bros.append(bro)
    return df

def init_matrix(name):
    df_team = pd.DataFrame(columns=bros + ["total"], index=bros)
    df_team.index.name = name
    df_team.total = 0
    df_team = zero_out(df_team)
    return df_team
    
def init_matches():
    cols = ["t1 p1", "t1 p2", " vs ", "t2 p1", "t2 p2", "-->", "winning team", "win by"]
    df_matches = pd.DataFrame(columns=cols)
    return df_matches

def generate():
    df_team = init_matrix("team")
    df_against = init_matrix("against")
    df_matches = init_matches()
    match_idx, last_team_num = initial_selection(df_matches, df_team, df_against)
    return df_team, df_against, df_matches, match_idx, last_team_num

def update_against(df_against, df_matches, df_idx):
    t1p1 = df_matches.loc[df_idx, "t1 p1"]
    t1p2 = df_matches.loc[df_idx, "t1 p2"]
    t2p1 = df_matches.loc[df_idx, "t2 p1"]
    t2p2 = df_matches.loc[df_idx, "t2 p2"]
    
    print(f"Updating against player={t1p1} vs {t2p1} & {t2p2}")
    
    df_against.loc[t1p1, t2p1] = df_against.loc[t1p1, t2p1] + 1
    df_against.loc[t1p1, t2p2] = df_against.loc[t1p1, t2p2] + 1
    df_against.loc[t2p1, t1p1] = df_against.loc[t2p1, t1p1] + 1
    df_against.loc[t2p2, t1p1] = df_against.loc[t2p2, t1p1] + 1

    df_against.loc[t1p2, t2p1] = df_against.loc[t1p2, t2p1] + 1
    df_against.loc[t1p2, t2p2] = df_against.loc[t1p2, t2p2] + 1
    df_against.loc[t2p1, t1p2] = df_against.loc[t2p1, t1p2] + 1
    df_against.loc[t2p2, t1p2] = df_against.loc[t2p2, t1p2] + 1
    

def update_team(df_team, current_team):
    bro1, bro2 = current_team[0], current_team[1]
    print(f"Adding team: {bro1} and {bro2}")
    df_team.loc[bro1, bro2] = df_team.loc[bro1, bro2] + 1
    df_team.loc[bro2, bro1] = df_team.loc[bro2, bro1] + 1    
    return []



# for first run populate all bros
def initial_selection(df_matches, df_team, df_against):
    if not df_matches.empty:
        return df_matches    
    bros_remaining = bros.copy()    
    import random    
    
    
    match_idx = 0
    last_team_num = 1
    
    current_team = []
    while bros_remaining:                
        for player_idx in [0, 1, 3, 4]:
            bro_to_add = bros_remaining.pop(random.randint(0, len(bros_remaining)-1))
            df_matches.loc[match_idx, df_matches.columns[player_idx]] = bro_to_add                        
            current_team.append(bro_to_add)            
            if player_idx in [1, 4]:
                current_team = update_team(df_team, current_team)                    
            if player_idx == 4:
                last_team_num = 2
                # full team
                update_against(df_against, df_matches, match_idx)
                match_idx += 1       
            elif player_idx == 1:
                last_team_num = 1
            
            if not bros_remaining:
                break        
        
        
    
    df_team["total"] = df_team[df_team.columns[:-1]].sum()
    df_against["total"] = df_against[df_against.columns[:-1]].sum()
    
    df_matches.loc[:, " vs "] = ""
    df_matches.loc[:, "-->"] = ""
    
    # effectively row and set of columns
    return match_idx, last_team_num


In [124]:
def get_players_minimize_teams(player):
    ser_next_team = df_team.loc[player].dropna()
    ser_next_team = ser_next_team.loc[ser_next_team.min() == ser_next_team]
    return set(ser_next_team.index)

def get_players_minimize_matches(df_team):
    return set(df_team.loc[df_team.total.min() == df_team.total].index)

    
def make_algorithmic_selection(df_matches, last_team_num, match_idx):
    # next selection
    p1_col = "t2 p1" if last_team_num == 1 else "t1 p1"
    p2_col = "t2 p2" if last_team_num == 1 else "t1 p2"
    if last_team_num == 1:
        # select team 2 by first tracking opponents
        p1_against = df_matches.loc[match_idx, "t1 p1"]
        p2_against = df_matches.loc[match_idx, "t1 p2"]
        ser_against_1 = df_against.loc[p1_against].dropna()    
        ser_against_2 = df_against.loc[p2_against].dropna()    
        #print(f"Against_1=[{p1_against}, {ser_against_1.min()}], Against_2=[{p2_against}, {ser_against_2.min()}]")
        avail_bros_1 = set(ser_against_1[:-1].loc[ser_against_1 == ser_against_1.min()].index)
        avail_bros_2 = set(ser_against_2[:-1].loc[ser_against_2 == ser_against_2.min()].index)
        avail_bros = list((avail_bros_1 & avail_bros_2) & get_players_minimize_matches(df_team))
        #print(f"Available bros after filtering on min: {len(avail_bros_1)}, {len(avail_bros_2)}, set={len(avail_bros)}")

        # setting this one based on 
        if not avail_bros:
            avail_bros = list(get_players_minimize_matches(df_team))
            
        first_player = avail_bros.pop(random.randint(0, len(avail_bros)-1))
        df_matches.loc[match_idx, p1_col] = first_player        
        
        # set second by minimizing teammates and also avail bros
        avail_bros = list(set(avail_bros) & get_players_minimize_teams(first_player))
        
        if not avail_bros:
            avail_bros = list(get_players_minimize_matches(df_team))
            
            
        second_player = avail_bros.pop(random.randint(0, len(avail_bros)-1))
        df_matches.loc[match_idx, p2_col] = second_player
        
        # update teams
        df_team.loc[first_player, second_player] = df_team.loc[first_player, second_player] + 1
        df_team.loc[second_player, first_player] = df_team.loc[second_player, first_player] + 1
        df_team["total"] = df_team[df_team.columns[:-1]].sum()
        
        # update against
        update_against(df_against, df_matches, match_idx)
        df_against["total"] = df_against[df_against.columns[:-1]].sum()
        
        
        #p1 = avail_bros.pop(
        
        return match_idx + 1, 2
    else:
        
        # select team 1
        # p1 - player with fewer matches
        avail_min_matches = list(get_players_minimize_matches(df_team))
        
        first_player = avail_min_matches.pop(random.randint(0, len(avail_min_matches)-1))        
        df_matches.loc[match_idx, p1_col] = first_player
        
        

        avail_players = list(set(avail_min_matches) & get_players_minimize_teams(first_player))
        
        
        second_player = avail_players.pop(random.randint(0, len(avail_players)-1))
        df_matches.loc[match_idx, p2_col] = second_player

        df_team.loc[first_player, second_player] = df_team.loc[first_player, second_player] + 1
        df_team.loc[second_player, first_player] = df_team.loc[second_player, first_player] + 1
        df_team["total"] = df_team[df_team.columns[:-1]].sum()

        return match_idx, 1
        #raise NotImplementedError()



In [134]:
df_team, df_against, df_matches, match_idx, last_team_num = generate()
while 1==1:
    match_idx, last_team_num = make_algorithmic_selection(df_matches, last_team_num, match_idx)
    if match_idx == 10:
        break
df_matches.loc[:, " vs "] = ""
df_matches.loc[:, "-->"] = ""
df_team["total"] = df_team[df_team.columns[:-1]].sum()
df_against["total"] = df_against[df_against.columns[:-1]].sum()
df_matches[df_matches.columns[:5]]

Adding team: Nick and Lacy
Adding team: Conway and Walker
Updating against player=Nick vs Conway & Walker
Adding team: Joe and Max
Adding team: Knopf and Thomas
Updating against player=Joe vs Knopf & Thomas
Adding team: Jon and Jungstein
Updating against player=Jon vs Joe & Walker
Updating against player=Nick vs Knopf & Max
Updating against player=Jon vs Thomas & Thomas
Updating against player=Conway vs Jungstein & Walker
Updating against player=Thomas vs Knopf & Nick
Updating against player=Conway vs Max & Joe
Updating against player=Nick vs Joe & Conway
Updating against player=Thomas vs Knopf & Jon


,t1 p1,t1 p2,vs,t2 p1,t2 p2
0,Nick,Lacy,,Conway,Walker
1,Joe,Max,,Knopf,Thomas
2,Jon,Jungstein,,Joe,Walker
3,Nick,Jungstein,,Knopf,Max
4,Jon,Lacy,,Thomas,Thomas
5,Conway,Thomas,,Jungstein,Walker
6,Thomas,Jon,,Knopf,Nick
7,Conway,Lacy,,Max,Joe
8,Nick,Max,,Joe,Conway
9,Thomas,Lacy,,Knopf,Jon


In [135]:
df_team

,Jungstein,Max,Walker,Knopf,Joe,Thomas,Lacy,Jon,Conway,Nick,total
team,,,,,,,,,,,
Jungstein,NaN,0,1,0,0,0,0,1,0,1,3
Max,0,NaN,0,1,2,0,0,0,0,1,4
Walker,1,0,NaN,0,1,0,0,0,1,0,3
Knopf,0,1,0,NaN,0,1,0,1,0,1,4
Joe,0,2,1,0,NaN,0,0,0,1,0,4
Thomas,0,0,0,1,0,NaN,1,1,1,0,4
Lacy,0,0,0,0,0,1,NaN,1,1,1,4
Jon,1,0,0,1,0,1,1,NaN,0,0,4
Conway,0,0,1,0,1,1,1,0,NaN,0,4


In [136]:
df_against

,Jungstein,Max,Walker,Knopf,Joe,Thomas,Lacy,Jon,Conway,Nick,total
against,,,,,,,,,,,
Jungstein,NaN,1,1,1,1,1,0,0,1,0,6
Max,1,NaN,0,1,1,1,1,0,2,1,8
Walker,1,0,NaN,0,0,1,1,1,1,1,6
Knopf,1,1,0,NaN,1,2,1,1,0,1,8
Joe,1,1,0,1,NaN,1,1,1,1,1,8
Thomas,1,1,1,2,1,NaN,2,3,0,1,12
Lacy,0,1,1,1,1,2,NaN,1,1,0,8
Jon,0,0,1,1,1,3,1,NaN,0,1,8
Conway,1,2,1,0,1,0,1,0,NaN,2,8
